# Generate synpuf disclosure risk

Compare synthetic PUFs trained from a 10% sample, both to the training set and a 10% holdout. Synthetic file (1) is from synthimpute random forests; (2) is from the synthpop R package.

## Setup

### Imports

In [1]:
import pandas as pd
import numpy as np
import synthimpute as si
import synpuf

/home/maxghenis/miniconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: During startup - 
  warnings.warn(x, RRuntimeWarning)
/home/maxghenis/miniconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Warning message:

  warnings.warn(x, RRuntimeWarning)
/home/maxghenis/miniconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: package 'RevoUtils' was built under R version 3.4.3 

  warnings.warn(x, RRuntimeWarning)


### Load data

In [2]:
# synth = pd.read_csv('~/Downloads/puf_synth_50p_sample.csv')
synthpop = pd.read_csv('~/Downloads/puf_synthpop_50p_sample.csv')
train = pd.read_csv('~/Downloads/puf_50p_sample_train.csv')
test = pd.read_csv('~/Downloads/puf_50p_sample_test.csv')

## Preprocessing

Drop calculated features used as seeds, and drop s006.

In [3]:
synthpop.drop(['S006'], axis=1, inplace=True)
synpuf.add_subtracted_features(train)
synpuf.add_subtracted_features(test)

Check pending https://github.com/MaxGhenis/synthimpute/issues/8.

In [4]:
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

## RF

Compare nearest standardized Euclidean distance.

Takes about 2min per 1,000 records, but up to 9 hours for all 80k.

In [13]:
%%time
nearest = si.nearest_synth_train_test(synthpop.iloc[:100], train, test)

Calculating nearest records to training set...
Calculating nearest records to test set...
CPU times: user 4.4 s, sys: 3.03 s, total: 7.43 s
Wall time: 7.12 s


In [10]:
%%time
nearest = si.nearest_synth_train_test(synthpop.iloc[:1000], train, test)

Calculating nearest records to training set...
Calculating nearest records to test set...
CPU times: user 1min 2s, sys: 41.1 s, total: 1min 43s
Wall time: 1min 49s


In [5]:
%%time
nearest = si.nearest_synth_train_test(synthpop.sample(frac=0.2), train, test)

Calculating nearest records to training set...


/home/maxghenis/miniconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:83: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Calculating nearest records to test set...
CPU times: user 11min 52s, sys: 8min 56s, total: 20min 48s
Wall time: 2h 3min 26s


In [16]:
%%time
nearest = si.nearest_synth_train_test(synthpop, train, test)

Calculating nearest records to training set...


KeyboardInterrupt: 

In [ ]:
nearest.to_csv('~/Downloads/nearest_synthpop_50p.csv')

## synthpop

In [10]:
%%time
nearest2 = si.nearest_synth_train_test(synthpop, train, test, BLOCKS, metric='euclidean')

Running block 1 of 14...
Running block 2 of 14...
Running block 3 of 14...
Running block 4 of 14...
Running block 5 of 14...
Running block 6 of 14...
Running block 7 of 14...
Running block 8 of 14...
Running block 9 of 14...
Running block 10 of 14...
Running block 11 of 14...
Running block 12 of 14...
Running block 13 of 14...
Running block 14 of 14...
Running block 1 of 14...
Running block 2 of 14...
Running block 3 of 14...
Running block 4 of 14...
Running block 5 of 14...
Running block 6 of 14...
Running block 7 of 14...
Running block 8 of 14...
Running block 9 of 14...
Running block 10 of 14...
Running block 11 of 14...
Running block 12 of 14...
Running block 13 of 14...
Running block 14 of 14...
CPU times: user 1min 42s, sys: 22.9 s, total: 2min 5s
Wall time: 1min 42s


In [11]:
nearest2.to_csv('~/Downloads/nearest_synthpop_samp.csv')